In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk import download as nltk_download
#from nltk.stem import TurkishStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import re

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/haber-snflandrma/data.csv


In [2]:
# Gerekli NLTK veri setlerini indir
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
stop_words = set(stopwords.words('turkish'))
# Türkçe stop words'leri yükleme


In [4]:
# Veri setini yükleme
data = pd.read_csv('/kaggle/input/haber-snflandrma/data.csv')



In [5]:
# 'Haber Gövdesi' sütununu temizleme
data['Haber Gövdesi'] = data['Haber Gövdesi'].str.replace('[^\w\s]', '')  # Özel karakterleri kaldırma
data['Haber Gövdesi'] = data['Haber Gövdesi'].apply(lambda x: x.lower())  # Küçük harfe dönüştürme


In [6]:
from snowballstemmer import TurkishStemmer
#from unicode_tr import unicode_tr
# Türkçe stop words'leri yükleme
stop_words = set(stopwords.words('turkish'))
# Türkçe için TurkishStemmer kullanma
stemmer = TurkishStemmer()


In [7]:
# Metin sütununu tokenize etme ve stemming işlemi için SnowballStemmer kullanma
#stop_words = set(stopwords.words('turkish'))
#stemmer = SnowballStemmer('turkish')  # Türkçe için SnowballStemmer kullanma

def preprocess_text(text):
    tokens = word_tokenize(text)  # Metni kelimelere ayırma
    filtered_tokens = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words]  # Stop words'leri kaldırma ve stemming
    return ' '.join(filtered_tokens)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Her sınıf için ayrı TF-IDF vektörlerini saklayacak bir sözlük oluşturun
tfidf_vectors = {}

# Her sınıf için TF-IDF vektörlerini hesaplayın
for category in data['Sınıf'].unique():
    # Belge koleksiyonunu seçilen sınıfa göre filtreleme
    documents_in_category = data[data['Sınıf'] == category]['Haber Gövdesi']
    
    # TF-IDF vektörlerini hesaplayın
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents_in_category)
    
    # TF-IDF vektörlerini sözlüğe kaydetme
    tfidf_vectors[category] = (vectorizer, tfidf_matrix)

# Örnek bir sınıfın TF-IDF vektörlerini yazdırma
for category, (vectorizer, tfidf_matrix) in tfidf_vectors.items():
    print(f"{category} sınıfının TF-IDF vektörleri:")
    terms = vectorizer.get_feature_names_out()
    print(f"Terimler: {terms}")
    print(f"TF-IDF matrisi: {tfidf_matrix}")
    print("\n")

Magazin sınıfının TF-IDF vektörleri:
Terimler: ['00' '000' '0000' ... '𝔊𝔯𝔦𝔪𝔢𝔰' '𝗗𝗨𝗬𝗚𝗨𝗞𝗔𝗬𝗔𝗞𝗨𝗠𝗔𝗥𝗞i' '𝚃𝚞𝚋𝚊𝚐𝚘𝚘𝚍𝚠𝚒𝚕𝚕𝚌𝚑𝚒𝚕𝚎']
TF-IDF matrisi:   (0, 155735)	0.06320826840175829
  (0, 997263)	0.020614259203883777
  (0, 498081)	0.04958977073347379
  (0, 440996)	0.06749112771876915
  (0, 864315)	0.030218492705903068
  (0, 295434)	0.06320826840175829
  (0, 330389)	0.06078107731414734
  (0, 145689)	0.0675036010963648
  (0, 947824)	0.0836531713350239
  (0, 289683)	0.05871350216934317
  (0, 644363)	0.057421753713541614
  (0, 752503)	0.06884029174427966
  (0, 803384)	0.0408764668610796
  (0, 299468)	0.05240590473632069
  (0, 81743)	0.04068498987331987
  (0, 406621)	0.037264881135893026
  (0, 291241)	0.08180246887342545
  (0, 355890)	0.0690583855747484
  (0, 299667)	0.051519851286331866
  (0, 294571)	0.032009796124174185
  (0, 60074)	0.04171134475506713
  (0, 220378)	0.0393796056107086
  (0, 947831)	0.08036695251959965
  (0, 980358)	0.06731956683525339
  (0, 404788)	0.046967024377732945
  :	:
  (489949

In [9]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Her sınıf için ayrı PCA dönüşümlerini saklayacak bir sözlük oluşturun
pca_transforms = {}

# PCA dönüşümlerini hesaplayın ve her sınıf için saklayın
for category, (vectorizer, tfidf_matrix) in tfidf_vectors.items():
    # PCA modelini oluşturun ve uygulayın
    pca = PCA(n_components=2)  # İki boyutlu uzaya indirgeme
    pca_result = pca.fit_transform(tfidf_matrix.toarray())
    
    # PCA dönüşümünü saklayın
    pca_transforms[category] = pca_result

# Her sınıf için PCA sonuçlarını görselleştirin
plt.figure(figsize=(10, 8))
for category, pca_result in pca_transforms.items():
    plt.scatter(pca_result[:, 0], pca_result[:, 1], label=category, alpha=0.5)
plt.title('TF-IDF Vektörlerinin PCA ile Görselleştirilmesi')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Her sınıf için ayrı PCA dönüşümlerini saklayacak bir sözlük oluşturun
pca_transforms = {}

# PCA dönüşümlerini hesaplayın ve her sınıf için saklayın
for category, (vectorizer, tfidf_matrix) in tfidf_vectors.items():
    # PCA modelini oluşturun ve uygulayın
    pca = PCA(n_components=2)  # İki boyutlu uzaya indirgeme
    pca_result = pca.fit_transform(tfidf_matrix.toarray())
    
    # PCA dönüşümünü saklayın
    pca_transforms[category] = pca_result

# Her sınıf için PCA sonuçlarını görselleştirin
plt.figure(figsize=(10, 8))
for category, pca_result in pca_transforms.items():
    plt.scatter(pca_result[:, 0], pca_result[:, 1], label=category, alpha=0.5)
plt.title('TF-IDF Vektörlerinin PCA ile Görselleştirilmesi')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# X ve Y değişkenlerine sütunları atama
X = data['Haber Gövdesi']
Y = data['Sınıf']

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Naive Bayes modelini oluşturma ve eğitme
nbModel = MultinomialNB()
nbModel.fit(X_train, Y_train)

# Test verileri üzerinde modelin performansını değerlendirme
Y_test_predicted = nbModel.predict(X_test)
accuracy = accuracy_score(Y_test, Y_test_predicted)
print('Test verileri üzerinde doğruluk: %.2f' % accuracy)